In [2]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
import time


In [2]:
#initiate a df to store the results
gitlab_df = pd.DataFrame()

In [ ]:
%timeit
#read each page and gather all public projects
for i in range(0,2510):
    g = requests.get(f"https://gitlab.com/api/v4/projects?per_page=20&page={i}")
    if(str(g)[11:14]=='200'):
        g_json = json.loads(g.text)
        g_df = pd.DataFrame(g_json)
        gitlab_df = gitlab_df.append(g_df)

In [4]:
#check records
gitlab_df = gitlab_df.reset_index()
gitlab_df.shape

(49940, 20)

In [6]:
#create list of ids to get licenses for each single project api
id_list = list(gitlab_df.id)
id_license_df = pd.DataFrame()

In [ ]:
%timeit
#gathers single project details including license
for i in id_list:
    g = requests.get(f"https://gitlab.com/api/v4/projects/{i}?license=True")
    if(str(g)[11:14]=='200'):
        g_json = json.loads(g.text)
        g_df = pd.json_normalize(g_json)
        id_license_df = id_license_df.append(g_df)

In [8]:
id_license_df = id_license_df.reset_index()
id_license_df

,index,id,description,name,name_with_namespace,path,path_with_namespace,created_at,default_branch,tag_list,...,namespace.kind,namespace.full_path,namespace.parent_id,namespace.avatar_url,namespace.web_url,license.key,license.name,license.nickname,license.html_url,license.source_url
0,0,29921595,,activity,zuitt-projects / b123 / s18 / activity,activity,zuitt-projects-chif/b123/s18/activity,2021-09-27T01:23:53.279Z,main,[],...,group,zuitt-projects-chif/b123/s18,13264943,None,https://gitlab.com/groups/zuitt-projects-chif/...,NaN,NaN,NaN,NaN,NaN
1,0,29921586,前端代码案例,qianduan,lixiang406 / qianduan,qianduan,lixiang406/qianduan,2021-09-27T01:23:37.386Z,main,[],...,user,lixiang406,None,https://secure.gravatar.com/avatar/5b0aac89a4a...,https://gitlab.com/lixiang406,NaN,NaN,NaN,NaN,NaN
2,0,29921581,,Debsigs,Todd Miller / Debsigs,debsigs,millertc/debsigs,2021-09-27T01:23:11.321Z,master,[],...,user,millertc,None,https://secure.gravatar.com/avatar/ce5ae12465b...,https://gitlab.com/millertc,gpl-2.0,GNU General Public License v2.0,GNU GPLv2,http://choosealicense.com/licenses/gpl-2.0/,None
3,0,29921571,https://cyberpirates.net,Cyber Pirates,Cyber Pirate / Cyber Pirates,cyber-pirates,CyberPirate98/cyber-pirates,2021-09-27T01:21:53.056Z,main,[],...,user,CyberPirate98,None,https://secure.gravatar.com/avatar/ed0761fa4f8...,https://gitlab.com/CyberPirate98,NaN,NaN,NaN,NaN,NaN
4,0,29921560,,gallerygaze,Shan Memon / gallerygaze,gallerygaze,skm3263/gallerygaze,2021-09-27T01:20:36.351Z,main,[],...,user,skm3263,None,https://secure.gravatar.com/avatar/993baa9c9c7...,https://gitlab.com/skm3263,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49911,0,29664275,Project with CI/CD Pipelines,project-with-ci-cd-10e299bf31f9984c,gitlab-qa-sandbox-group / qa-test-2021-09-16-1...,project-with-ci-cd-10e299bf31f9984c,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,2021-09-16T12:38:50.836Z,main,[],...,group,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,4909902,None,https://gitlab.com/groups/gitlab-qa-sandbox-gr...,NaN,NaN,NaN,NaN,NaN
49912,0,29664274,Project with repository,project-with-code-514cf462051d02f0,gitlab-qa-sandbox-group / qa-test-2021-09-16-1...,project-with-code-514cf462051d02f0,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,2021-09-16T12:38:49.890Z,main,[],...,group,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,4909902,None,https://gitlab.com/groups/gitlab-qa-sandbox-gr...,NaN,NaN,NaN,NaN,NaN
49913,0,29664273,project for git clone tests,project-with-code-6992a6b69bfa67d2,gitlab-qa-sandbox-group / qa-test-2021-09-16-1...,project-with-code-6992a6b69bfa67d2,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,2021-09-16T12:38:45.691Z,main,[],...,group,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,4909902,None,https://gitlab.com/groups/gitlab-qa-sandbox-gr...,NaN,NaN,NaN,NaN,NaN
49914,0,29664272,My awesome project,upload-file-project-7f73a943843be906,gitlab-qa-sandbox-group / qa-test-2021-09-16-1...,upload-file-project-7f73a943843be906,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,2021-09-16T12:38:45.471Z,main,[],...,group,gitlab-qa-sandbox-group/qa-test-2021-09-16-12-...,4909902,None,https://gitlab.com/groups/gitlab-qa-sandbox-gr...,NaN,NaN,NaN,NaN,NaN


In [26]:
#save those with license int oa dataframe
with_license = id_license_df[(id_license_df['license.name'].notnull()) | (id_license_df['license.name'].notna())]

In [28]:
#save as excel
with_license = with_license[with_license['license.name'].str.lower()!='other'].drop(columns=['index'])
with_license.to_excel('gitlab_withlicense_public.xlsx')

In [17]:
#create df to store contributor info
contributor_df = pd.DataFrame()

In [13]:
#read the created file instead of querying api again if required.
with_lic_copy = pd.read_excel('gitlab_withlicense_public.xlsx').drop(columns=['Unnamed: 0'])

In [ ]:
%timeit
#https://docs.gitlab.com/ee/api/repositories.html
#get contributor info for each project
for i in with_lic_copy.id:
    g = requests.get(f"https://gitlab.com/api/v4/projects/{i}/repository/contributors")
    if(str(g)[11:14]=='200'):
        print(i)
        g_json = json.loads(g.text)
        g_df = pd.json_normalize(g_json)
        g_df['id'] = i
        contributor_df = contributor_df.append(g_df)

In [33]:
#save to excel
contributor_df.to_excel('gitlab_withlicense_public_contributors.xlsx')